[Reference](https://trojrobert.medium.com/a-guide-for-deploying-machine-learning-model-with-streamlit-vs-gradio-563a0b2dc1bd)

In [3]:
pip install gradio

     |████████████████████████████████| 649 kB 5.4 MB/s 
     |████████████████████████████████| 210 kB 59.6 MB/s 
     |████████████████████████████████| 1.1 MB 40.0 MB/s 
     |████████████████████████████████| 84 kB 2.7 MB/s 
     |████████████████████████████████| 2.0 MB 39.0 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 255 kB 63.7 MB/s 
     |████████████████████████████████| 53 kB 2.4 MB/s 
     |████████████████████████████████| 271 kB 62.4 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 144 kB 64.1 MB/s 
     |████████████████████████████████| 58 kB 6.8 MB/s 
     |████████████████████████████████| 10.9 MB 42.6 MB/s 
     |████████████████████████████████| 79 kB 9.1 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 61 kB 489 kB/s 
     |████████████████████████████████| 856 kB 54.2 MB/s 
     |████████████████████████

In [1]:
# https://github.com/trojrobert/deploying_image_classification/blob/main/gradio_image_classification/app.py
import requests

import gradio as gr

from PIL import Image
import torch
from torchvision import models, transforms 

def get_prediction(input_image, model, labels):
    tensor = transform_image(image=input_image)
    outputs = model.forward(tensor)
    _, y_hat = outputs.max(1)
    y_hat = y_hat.numpy()

    with torch.no_grad():
        prediction = torch.nn.functional.softmax(model(tensor)[0], dim=0)
    
    return labels[y_hat[0]]


def transform_image(image):
    """ Transform image to fit model
    Args:
        image (image): Input image from the user
    Returns:
        tensor: transformed image 
    """
    transformation = transforms.Compose([transforms.ToPILImage(),
                                        transforms.Resize(255),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize(
                                            [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])])
    return transformation(image).unsqueeze(0)


def load_model_and_labels():
    # Make sure to pass `pretrained` as `True` to use the pretrained weights:
    # Since we are using our model only for inference, switch to `eval` mode:
    model = models.resnet18(pretrained=True).eval()

    # Download human-readable labels for ImageNet.
    response = requests.get("https://git.io/JJkYN")
    labels = response.text.split("\n")
   
    return model, labels


def main(input_image):
    
    model, labels = load_model_and_labels()
    #image = image.reshape(1, -1)

    prediction = get_prediction(input_image, model, labels)
    return prediction

iface = gr.Interface(
    fn=main,
    inputs=gr.inputs.Image(shape=(224, 224)),
    outputs=gr.outputs.Label(num_top_classes=3),
    title="Predict objects in an image",
    description="This application can predict objects in an image , but works best when only one object is in the image",
    live=True,
    interpretation="default",
    capture_session=True
)


if __name__ == '__main__':
    iface.launch(share=True)

In [4]:
pip install streamlit

     |████████████████████████████████| 9.7 MB 5.5 MB/s 
     |████████████████████████████████| 111 kB 74.8 MB/s 
     |████████████████████████████████| 164 kB 41.3 MB/s 
     |████████████████████████████████| 76 kB 3.3 MB/s 
     |████████████████████████████████| 4.3 MB 39.4 MB/s 
     |████████████████████████████████| 181 kB 45.8 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 128 kB 46.6 MB/s 
     |████████████████████████████████| 793 kB 49.1 MB/s 
     |████████████████████████████████| 380 kB 40.3 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=ab295322cd27742808f4acb6f723b2fc881970d49e7365700b6b6de095c2d9c9
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      S

In [2]:
# https://github.com/trojrobert/deploying_image_classification/blob/main/streamlit_image_classification/app.py
import requests

import streamlit as st

from PIL import Image
import torch
from torchvision import models, transforms 

def get_prediction(input_image, model, labels):
    """[summary]
    Args:
        image (): input image
        model ([type]): pytorch model 
        labels([type]): ImageNet readable labels
    Returns:
        [type]: Predictions from the labels
    """

    tensor = transform_image(image=input_image)
    outputs = model.forward(tensor)
    _, y_hat = outputs.max(1)
    y_hat = y_hat.numpy()

    with torch.no_grad():
        prediction = torch.nn.functional.softmax(model(tensor)[0], dim=0)
    
    return labels[y_hat[0]]


def transform_image(image):
    """ Transform image to fit model
    Args:
        image (image): Input image from the user
    Returns:
        tensor: transformed image 
    """
    transformation = transforms.Compose([transforms.Resize(255),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize(
                                            [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])])
    return transformation(image).unsqueeze(0)


@st.cache
def load_model_and_labels():
    # Make sure to pass `pretrained` as `True` to use the pretrained weights:
    # Since we are using our model only for inference, switch to `eval` mode:
    model = models.resnet18(pretrained=True).eval()

    #download human-readable label for ImageNet
    response = requests.get("https://git.io/JJkYN")
    labels = response.text.split("\n")
    return model, labels


def main():
    
    st.title("Predict objects in an image")
    st.write("This application can predict objests in an image , but works best when only one object is in the image")

    model, labels = load_model_and_labels()
    
    #Create a UI component to read image
    image_file  = st.file_uploader("Upload an image", type=['jpg', 'jpeg', 'png'])

    if image_file:
        
        #divide your interface to two parts
        left_column, right_column = st.columns(2)
        left_column.image(image_file, caption="Uploaded image", use_column_width=True)
        input_image = Image.open(image_file)

        #create a UI component to create a button
        pred_button = st.button("Predict")
        
        
        if pred_button:

            prediction = get_prediction(input_image, model, labels)
            right_column.title("Prediction")
            right_column.write(prediction)

if __name__ == '__main__':
    main()

In [6]:
# !python your_script.py

In [5]:
# !streamlit run your_script.py